In [1]:
# TESTING This notebook
print("TESTING This notebook")

TESTING This notebook


In [2]:
from flexneuart import configure_classpath

configure_classpath()

In [3]:
COLLECTION_DIR='/home/ubuntu/efs/capstone/data/orcas/orcas'

In [4]:
from flexneuart.indexing import *

vars(get_index_query_data_dirs(f'{COLLECTION_DIR}/input_data_main'))

Checking sub-directory: queries
Found indexable data file: /home/ubuntu/efs/capstone/data/orcas/orcas/input_data_main/queries/AnswerFields.jsonl.gz


{'index_subdirs': ['queries'],
 'query_subdirs': [],
 'data_file_name': 'AnswerFields.jsonl.gz'}

In [6]:
# change False to True if inverse index has not been created and run this cell

if False:
    create_lucene_index(input_data_dir=f'{COLLECTION_DIR}/input_data_main',
                        output_dir_name=f'{COLLECTION_DIR}/lucene_index',
                        index_field_name='text')

Checking sub-directory: queries
Found indexable data file: /home/ubuntu/efs/capstone/data/orcas/orcas/input_data_main/queries/AnswerFields.jsonl.gz


In [ ]:
# change False to True if forward index has not been created and run this cell

index_field_names=['text_raw']
index_field_types=['textRaw']

if False:
    for field_name, field_type in zip(index_field_names, index_field_types):
        create_forward_index(input_data_dir=f'{COLLECTION_DIR}/input_data_main',
                            output_dir_name=f'{COLLECTION_DIR}/forward_index',
                            index_field_name=field_name,
                            field_type=field_type,
                            index_type=FORWARD_INDEX_TYPE_DATA_DICT)

Checking sub-directory: queries
Found indexable data file: /home/ubuntu/efs/capstone/data/orcas/orcas/input_data_main/queries/AnswerFields.jsonl.gz


#### retrieval

In [5]:
from flexneuart.retrieval import create_featextr_resource_manager

# create a resource manager
resource_manager=create_featextr_resource_manager(resource_root_dir=f'{COLLECTION_DIR}/',
                                                  fwd_index_dir='forward_index')

In [6]:
from flexneuart.retrieval.cand_provider import *
# create a candidate provider/generator
cand_prov = create_cand_provider(resource_manager, PROVIDER_TYPE_LUCENE, 
                                 f'lucene_index',
                                 add_config_file='exper_desc/lucene_conf.json')

from flexneuart.retrieval.fwd_index import get_forward_index
# get forward index
raw_indx = get_forward_index(resource_manager, 'text_raw')

In [7]:
from flexneuart.text_proc.parse import KrovetzStemParser
STEMMER_NO_STOPWORDS=KrovetzStemParser(stop_words=[])

def jaccard(text1, text2, text_proc):
    set1 = set(text_proc(text1.lower()).split())
    set2 = set(text_proc(text2.lower()).split())
    
    len1 = len(set1)
    len2 = len(set2)
    mxlen = max(len1, len2)
    if not mxlen:
        return 0
    return len(set1 & set2) / len(set1 | set2)

In [12]:
orig_queries = ["who published romeo and juliet", "which country is london in"]

json_array = []

K = 5 # total no of new queries cap needed

for orig_query in orig_queries:
    # An example of running a text query
    query_res = run_text_query(cand_prov, 100, orig_query)
    # query_res
    total_new_queries_added = 0
    new_queries_added = []
    for e in query_res[1]: # e = CandidateEntry(doc_id='8206135', score=18.488903045654297),
        reform_query = raw_indx.get_doc_text_raw(e.doc_id)
        simil = jaccard(orig_query, reform_query, STEMMER_NO_STOPWORDS)
        if(orig_query==reform_query):
            continue
        if simil >= 0.6 and simil<1:
            print(e.doc_id, simil, reform_query)
            print('=============')
            new_queries_added.append((e.doc_id, simil, reform_query))
            total_new_queries_added +=1
            json_array.append({'orig_qry_text' : orig_query, 'replacement_text' : reform_query, 'sim_score': simil})
        if(total_new_queries_added==K):
            break


8206135 0.6666666666666666 who composed romeo and juliet
8950310 0.6666666666666666 who composed romeo and juliet?
3797844 0.6666666666666666 who marries romeo and juliet
3502991 0.6666666666666666 who wrote romeo and juliet
11986085 0.6 juliet romeo and juliet
4339582 0.8 london in which country
11561458 0.6666666666666666 london is in which continent
3598845 0.6666666666666666 london is in which county
7095800 0.6666666666666666 which county is london in
11563909 0.6666666666666666 abidjan is in which country


In [13]:
json_array

[{'orig_qry_text': 'who published romeo and juliet',
  'replacement_text': 'who composed romeo and juliet',
  'sim_score': 0.6666666666666666},
 {'orig_qry_text': 'who published romeo and juliet',
  'replacement_text': 'who composed romeo and juliet?',
  'sim_score': 0.6666666666666666},
 {'orig_qry_text': 'who published romeo and juliet',
  'replacement_text': 'who marries romeo and juliet',
  'sim_score': 0.6666666666666666},
 {'orig_qry_text': 'who published romeo and juliet',
  'replacement_text': 'who wrote romeo and juliet',
  'sim_score': 0.6666666666666666},
 {'orig_qry_text': 'who published romeo and juliet',
  'replacement_text': 'juliet romeo and juliet',
  'sim_score': 0.6},
 {'orig_qry_text': 'which country is london in',
  'replacement_text': 'london in which country',
  'sim_score': 0.8},
 {'orig_qry_text': 'which country is london in',
  'replacement_text': 'london is in which continent',
  'sim_score': 0.6666666666666666},
 {'orig_qry_text': 'which country is london in

In [14]:
import json
# write op file
outputfilename = 'temp.json'
with open(outputfilename, 'w') as outfile:
    json.dump(json_array, outfile)

In [15]:
# read ip file
with open(outputfilename, 'r') as outfile:
    json_array_loaded = json.load(outfile)
print(type(json_array_loaded))
print(json_array_loaded)

<class 'list'>
[{'orig_qry_text': 'who published romeo and juliet', 'replacement_text': 'who composed romeo and juliet', 'sim_score': 0.6666666666666666}, {'orig_qry_text': 'who published romeo and juliet', 'replacement_text': 'who composed romeo and juliet?', 'sim_score': 0.6666666666666666}, {'orig_qry_text': 'who published romeo and juliet', 'replacement_text': 'who marries romeo and juliet', 'sim_score': 0.6666666666666666}, {'orig_qry_text': 'who published romeo and juliet', 'replacement_text': 'who wrote romeo and juliet', 'sim_score': 0.6666666666666666}, {'orig_qry_text': 'who published romeo and juliet', 'replacement_text': 'juliet romeo and juliet', 'sim_score': 0.6}, {'orig_qry_text': 'which country is london in', 'replacement_text': 'london in which country', 'sim_score': 0.8}, {'orig_qry_text': 'which country is london in', 'replacement_text': 'london is in which continent', 'sim_score': 0.6666666666666666}, {'orig_qry_text': 'which country is london in', 'replacement_text

In [7]:
# example runs -->
# orig_query = "who published romeo and juliet"

# # An example of running a text query
# query_res = run_text_query(cand_prov, 100, orig_query)
# query_res

(4274,
 [CandidateEntry(doc_id='8206135', score=18.488903045654297),
  CandidateEntry(doc_id='8950310', score=18.488903045654297),
  CandidateEntry(doc_id='3797844', score=18.488903045654297),
  CandidateEntry(doc_id='3502991', score=18.488903045654297),
  CandidateEntry(doc_id='8827722', score=18.292387008666992),
  CandidateEntry(doc_id='11975473', score=17.66094970703125),
  CandidateEntry(doc_id='10862739', score=17.66094970703125),
  CandidateEntry(doc_id='5517033', score=16.903972625732422),
  CandidateEntry(doc_id='9328272', score=16.903972625732422),
  CandidateEntry(doc_id='3342849', score=16.903972625732422),
  CandidateEntry(doc_id='11563115', score=16.903972625732422),
  CandidateEntry(doc_id='10428870', score=16.903972625732422),
  CandidateEntry(doc_id='3703539', score=16.903972625732422),
  CandidateEntry(doc_id='11494031', score=16.903972625732422),
  CandidateEntry(doc_id='4593336', score=16.903972625732422),
  CandidateEntry(doc_id='7339807', score=16.903972625732422)